In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import os
%matplotlib inline

warnings.filterwarnings("ignore") #, category=DeprecationWarning)

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [3]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

In [4]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [5]:
# df = pd.read_pickle('data/kickstarter_data.pkl')
# TODO determine whie ID shows up but is not called - index issue?
df = pd.read_sql_query('''SELECT main_category, main_category_code, currency, currency_code, 
                        deadline, usd_goal_real, usd_pledged_real, launched, state, state_code, 
                        country,country_code FROM kickstarter_data''',engine)
pd.read_sql_query('''SELECT * FROM kickstarter_data LIMIT 5''',engine)

,idx,ID,name,category,main_category,currency,deadline,goal,launched,pledged,...,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved,main_category_code,country_code,currency_code,state_code
0,1031685482,1031685482,You Sparkle Inside: A Kids' Book + Commissions...,Children's Books,Publishing,USD,2018-01-02,2000.0,2017-12-06,6083.00,...,US,1054.0,6083.00,2000.0,27,304.150000,12,21,13,1
1,1176736360,1176736360,THE NEW BILLY WYLDER RECORD,Indie Rock,Music,USD,2018-01-02,10000.0,2017-11-30,11169.56,...,US,650.0,11169.56,10000.0,33,111.695600,10,21,13,1
2,1894703097,1894703097,Driftwood Holly ***The Venice Project***,Indie Rock,Music,EUR,2018-01-02,25000.0,2017-11-28,25417.20,...,DE,0.0,30615.02,30112.5,35,101.668809,10,5,4,1
3,311863059,311863059,On a Limb's debut album,Jazz,Music,USD,2018-01-02,1000.0,2017-12-09,1743.00,...,US,0.0,1743.00,1000.0,24,174.300000,10,21,13,1
4,2076121120,2076121120,Briggs Barbecue Sauces,Small Batch,Food,USD,2018-01-02,200000.0,2017-11-03,1.00,...,US,0.0,1.00,200000.0,60,0.000500,7,21,13,0


In [6]:
pd.read_sql_query('''SELECT count(name), SUM(usd_pledged_real) as USD_Pledged,SUM(usd_goal_real) as USD_Goal FROM kickstarter_data''',engine)

,count,usd_pledged,usd_goal
0,19291,2.627727e+08,8.743479e+08


In [7]:
pd.read_sql_query('''SELECT state,COUNT(state) as count, SUM(usd_goal_real) as usd_goal, SUM(usd_pledged_real) as usd_pledged FROM kickstarter_data GROUP BY state ORDER BY usd_pledged DESC LIMIT 10''',engine)

,state,count,usd_goal,usd_pledged
0,successful,8368,8.123888e+07,2.493829e+08
1,failed,10923,7.931090e+08,1.338983e+07


In [8]:
pd.read_sql_query('''SELECT main_category,COUNT(main_category) as count, SUM(usd_goal_real) as usd_goal,SUM(usd_pledged_real) as usd_pledged FROM kickstarter_data Group By main_category ORDER BY usd_pledged DESC LIMIT 5''',engine)

,main_category,count,usd_goal,usd_pledged
0,Games,2541,4.814846e+07,70608172.84
1,Design,2127,5.219732e+07,66428025.84
2,Technology,2081,1.065280e+08,56029162.80
3,Film & Video,1943,3.962250e+08,13787683.40
4,Publishing,2005,2.271882e+07,10911971.51


In [9]:
pd.read_sql_query('''SELECT category,COUNT(category) as count, SUM(usd_goal_real) as usd_goal,SUM(usd_pledged_real) as usd_pledged FROM kickstarter_data Group By category ORDER BY usd_pledged DESC LIMIT 5''',engine)

,category,count,usd_goal,usd_pledged
0,Tabletop Games,1281,15384499.07,59007489.07
1,Product Design,1675,34575051.56,54426113.02
2,Hardware,175,10072078.60,11177130.20
3,Design,238,4752523.21,9818868.51
4,Wearables,98,4006337.79,8721065.16


In [10]:
pd.read_sql_query('''SELECT country, COUNT(country) as count,SUM(usd_goal_real) as usd_goal,SUM(usd_pledged_real)  as usd_pledged FROM kickstarter_data GROUP BY country ORDER BY usd_pledged DESC LIMIT 5''',engine)

,country,count,usd_goal,usd_pledged
0,US,12204,6.723174e+08,1.872663e+08
1,GB,2207,4.608401e+07,1.871978e+07
2,CA,962,3.122999e+07,7.231933e+06
3,DE,519,1.847728e+07,6.270310e+06
4,NL,216,4.211652e+06,5.824917e+06


In [11]:
pd.read_sql_query('''SELECT currency,COUNT(currency) as count,SUM(usd_goal_real) as usd_goal,SUM(usd_pledged_real) as usd_pledged FROM kickstarter_data GROUP BY currency ORDER BY usd_pledged DESC LIMIT 5''',engine)

,currency,count,usd_goal,usd_pledged
0,USD,12204,6.723174e+08,1.872663e+08
1,EUR,1941,7.315434e+07,2.764281e+07
2,GBP,2207,4.608401e+07,1.871978e+07
3,CAD,962,3.122999e+07,7.231933e+06
4,AUD,581,1.419635e+07,5.774700e+06
